In this notebook we will index the french dump version of the wikipedia dataset, and use with the retrieval for our qa/system

In [ ]:
import numpy as np
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers

In [ ]:
from datasets import load_dataset

In [ ]:
wiki_corpus = load_dataset("wikipedia", "20220301.fr", split="train")

In [ ]:
wiki_corpus = wiki_corpus.shuffle(seed=42)

In [ ]:
sample_wiki = wiki_corpus.shard(100, index=0)

In [ ]:
sample_doc_dict = sample_wiki[1]

In [ ]:
from haystack.schema import Document

In [ ]:
docs = Document.from_dict(sample_doc_dict, field_map={"text": "content"})

#### Building the Elastic Search Index

In [2]:
from haystack.utils import clean_wiki_text, convert_files_to_docs

In [3]:
from haystack.errors import HaystackError
from haystack.schema import Document
from typing import List, Optional, Generator, Set, Union, Callable, Dict
from copy import deepcopy
from haystack.nodes import PreProcessor
import re

In [4]:
from gensim.utils import deaccent

def remove_accents(document):
    input_without_accent = deaccent(document)
    return input_without_accent

In [5]:
async def convert_wiki_article_to_docs(
    item: dict,
    clean_func: Optional[Callable] = None,
    split_paragraphs: bool = False,
) -> List[Document]:
    """
    item

    :param items: dict of items
    :param clean_func: a custom cleaning function that gets applied to each doc (input: str, output:str)
    :param split_paragraphs: split text in paragraphs.
    """
    documents = []
    processed = 0
    text = item.get("text")
    text = remove_accents(text)
    if clean_func:
        text = clean_func(text)
    if split_paragraphs:
        for para in text.split("\n"):
            if 200 <= len(para.strip()) <= 2000:
                # just pick the paragraph with length between 50 and 1500
                processed += 1
                doc = Document(content=para, meta={"title": item.get("title")}, id=item.get("id"))
                documents.append(doc)
            else:
                continue
    else:
        processed += 1
        doc = Document(content=text, meta={"title": item.get("title")}, id=item.get("id"))
        documents.append(doc)
    
    return documents

In [ ]:
import asyncio

#### Saving the Document in the retriever

In [6]:
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
from haystack.document_stores import ElasticsearchDocumentStore


In [7]:
INDEX_NAME = 'fr-wikipedia'

In [8]:

document_store = ElasticsearchDocumentStore(index=INDEX_NAME, recreate_index=False, analyzer="french")

/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
INFO - haystack.document_stores.elasticsearch -  Index 'fr-wikipedia' deleted.
INFO - haystack.document_stores.elasticsearch -  Index 'label' deleted.


In [9]:
def split_list_in_chunck(list_, len_):
    chunks = [list_[i:i+len_] for i in range(0, len(list_), len_)]
    return chunks

In [10]:
from elasticsearch.helpers import bulk, parallel_bulk, scan

In [21]:
from collections import deque

In [11]:

def write_documents_parallel(
        self,
        documents: Union[List[dict], List[Document]],
        index: Optional[str] = None,
        batch_size: int = 10_000,
        duplicate_documents: Optional[str] = None,
        headers: Optional[Dict[str, str]] = None,
    ):
        """
        Indexes documents for later queries in Elasticsearch./ an update for the index that use parralell bluk

        Behaviour if a document with the same ID already exists in ElasticSearch:
        a) (Default) Throw Elastic's standard error message for duplicate IDs.
        b) If `self.update_existing_documents=True` for DocumentStore: Overwrite existing documents.
        (This is only relevant if you pass your own ID when initializing a `Document`.
        If don't set custom IDs for your Documents or just pass a list of dictionaries here,
        they will automatically get UUIDs assigned. See the `Document` class for details)

        :param documents: a list of Python dictionaries or a list of Haystack Document objects.
                          For documents as dictionaries, the format is {"content": "<the-actual-text>"}.
                          Optionally: Include meta data via {"content": "<the-actual-text>",
                          "meta":{"name": "<some-document-name>, "author": "somebody", ...}}
                          It can be used for filtering and is accessible in the responses of the Finder.
                          Advanced: If you are using your own Elasticsearch mapping, the key names in the dictionary
                          should be changed to what you have set for self.content_field and self.name_field.
        :param index: Elasticsearch index where the documents should be indexed. If not supplied, self.index will be used.
        :param batch_size: Number of documents that are passed to Elasticsearch's bulk function at a time.
        :param duplicate_documents: Handle duplicates document based on parameter options.
                                    Parameter options : ( 'skip','overwrite','fail')
                                    skip: Ignore the duplicates documents
                                    overwrite: Update any existing documents with the same ID when adding documents.
                                    fail: an error is raised if the document ID of the document being added already
                                    exists.
        :param headers: Custom HTTP headers to pass to elasticsearch client (e.g. {'Authorization': 'Basic YWRtaW46cm9vdA=='})
                Check out https://www.elastic.co/guide/en/elasticsearch/reference/current/http-clients.html for more information.
        :raises DuplicateDocumentError: Exception trigger on duplicate document
        :return: None
        """
        print("I am inside the function")
        if index and not self.client.indices.exists(index=index, headers=headers):
            self._create_document_index(index, headers=headers)

        if index is None:
            index = self.index
        duplicate_documents = duplicate_documents or self.duplicate_documents
        assert (
            duplicate_documents in self.duplicate_documents_options
        ), f"duplicate_documents parameter must be {', '.join(self.duplicate_documents_options)}"

        field_map = self._create_document_field_map()
        document_objects = [Document.from_dict(d, field_map=field_map) if isinstance(d, dict) else d for d in documents]
        document_objects_chunk = split_list_in_chunck(document_objects, 9990)
        document_objects_deduplicated = []
        for doc_chunk in document_objects_chunk:           
            chunk_deduplicated = self._handle_duplicate_documents(documents=doc_chunk, 
                                                                  index=index, 
                                                                  duplicate_documents=duplicate_documents, 
                                                                  headers=headers)
            document_objects_deduplicated.extend(chunk_deduplicated)
        documents_to_index = []
        # split the document in chunk of 10000
        for doc in tqdm(document_objects_deduplicated):
            _doc = {
                "_op_type": "index" if duplicate_documents == "overwrite" else "create",
                "_index": index,
                **doc.to_dict(field_map=self._create_document_field_map()),
            }  # type: Dict[str, Any]

            # cast embedding type as ES cannot deal with np.array
            if _doc[self.embedding_field] is not None:
                if type(_doc[self.embedding_field]) == np.ndarray:
                    _doc[self.embedding_field] = _doc[self.embedding_field].tolist()

            # rename id for elastic
            _doc["_id"] = str(_doc.pop("id"))

            # don't index query score and empty fields
            _ = _doc.pop("score", None)
            _doc = {k: v for k, v in _doc.items() if v is not None}

            # In order to have a flat structure in elastic + similar behaviour to the other DocumentStores,
            # we "unnest" all value within "meta"
            if "meta" in _doc.keys():
                for k, v in _doc["meta"].items():
                    _doc[k] = v
                _doc.pop("meta")
            documents_to_index.append(_doc)
        documents_to_index_chunk = split_list_in_chunck(documents_to_index, 9990)
        for doc_chunk in tqdm(documents_to_index_chunk):
            pb_ = parallel_bulk(self.client, 
                              doc_chunk, 
                              chunk_size=10000, 
                              thread_count=8, 
                              queue_size=8,
                              refresh=self.refresh_type, 
                              headers=headers)
            deque(pb_, maxlen=0)
            documents_to_index = []

In [12]:
document_store.write_documents_parallel = write_documents_parallel

In [13]:
%%script false --no-raise-error
all_docs = []
for i in range(0, 100):
    shard = wiki_corpus.shard(100, index=i)
    with tqdm(total=shard.shape[0]) as pbar:
        docs_in_shard = tqdm_asyncio.gather(*[convert_wiki_article_to_docs(item, clean_func=clean_wiki_text, split_paragraphs=True) for item in shard])
        all_docs.append(docs_in_shard)
    print("done with shard ", i)

In [14]:
from  functools import reduce
from operator import iconcat

In [15]:
%%script false --no-raise-error
with tqdm(total=len(all_docs)) as pbar:
     scan_results = await tqdm_asyncio.gather(*all_docs[90:100])

In [16]:
%%script false --no-raise-error
scan_results = reduce(iconcat, scan_results, [])
scan_results = reduce(iconcat, scan_results, [])

With our document indexted int the elastic searh we can search , use the piaf dataset which have question with answers without paragraph and leverage them.

#### Reading document from File

The firt approach have saved the document in differents files in our project, but now we need to read them to read those files for the second test

In [17]:
import json
from pathlib import Path
DATA_PATH = Path.cwd().parent.joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"
wikipedia_dump = DATA_PATH.joinpath("wikipedia")
assert wikipedia_dump.exists()

In [25]:
 saved_files = []

[PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_97.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_57.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_12.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_73.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_36.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_71.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_34.jsonl'),
 PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/wikipedia/dump_55.j

In [27]:
%%script false --no-raise-error
for index, file in enumerate(wikipedia_dump.iterdir()):
    if file not in saved_files:
        with open(file, "r") as buffer:
            docs = []
            for index_, data in enumerate(buffer.readlines()):
                id_ = f"{index}{index_}"
                json_data = json.loads(json.loads(data))
                json_data["id"] = id_
                document = Document.from_dict(json_data)
                docs.append(document)
            write_documents_parallel(document_store, docs)
            print(f"done saving the {index}th batch document")
        saved_files.append(file)
    else:
        print("I have already processed that file")

#### Writing Congo News Dataset

In [139]:
DATA_PATH

PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data')

In [140]:

DRC_NEWS_DATA_PATH = DATA_PATH.joinpath("corpus", "drc-news-txt")

In [141]:
import pandas as pd

In [142]:
data_file_path = DATA_PATH.joinpath("corpus", "raw", 'drc-news-raws.csv')

In [143]:
cd_news_data = pd.read_csv(data_file_path, names=["content", "posted_at"])

In [144]:
cd_news_data = cd_news_data.fillna(value="")
cd_news_data.head()

,content,posted_at
0,Les membres de la Commission tarifaire viennent de proposer des mesures néce...,2022-09-05 00:00:00
1,Les membres de la Commission tarifaire sont en session extraordinaire d...,2022-04-05 00:00:00
2,"Vodacom Congo vient de signer un partenariat avec Kinshasa Digital Academy, ...",2022-04-23 00:00:00
3,"Le sélectionneur des Léopards de la RDC, Hectór Cúper est attendu à Kinshasa...",2022-03-05 00:00:00
4,Le protocole d’accord était déjà signé entre la RDC et la compagnie aérienne...,2022-11-05 00:00:00


In [180]:
pd.to_datetime(cd_news_data.posted_at, errors="coerce").dropna().max()

Timestamp('2022-12-05 00:00:00')

In [145]:
from haystack.nodes import TextConverter

In [146]:
from haystack.schema import Document
from secrets import token_hex

# @Todo: this is not working now , it was supposed to save the document to dataframe
def get_document_from_text(row):
    """numpy row with the text and the date of the post

    Args:
        row (_type_): _description_

    Returns:
        _type_: _description_
    """
    text = row[0].replace(u'\xa0', u' ')
    for paragraph in text.split("   "):
        if not paragraph.strip():  # skip empty paragraphs
            continue
        return Document(content=paragraph, meta={"posted_at":row[1] if row[1] else "" })

In [147]:
all_cd_news_docs = cd_news_data.apply(get_document_from_text, axis="columns")

In [148]:
len(all_cd_news_docs)

140638

In [149]:
all_cd_news_docs = all_cd_news_docs.dropna().to_list()

In [150]:
from haystack.errors import HaystackError
from haystack.schema import Document
from typing import List, Optional, Generator, Set, Union
from copy import deepcopy
from haystack.nodes import PreProcessor

class CustomPreProcessor(PreProcessor):
    def __init__(self, custom_preprocessor=None, **kwargs):
        super().__init__(**kwargs)
        self.custom_preprocessor = custom_preprocessor
    def clean(
        self,
        document: Union[dict, Document],
        clean_whitespace: bool,
        clean_header_footer: bool,
        clean_empty_lines: bool,
        remove_substrings: List[str],
        id_hash_keys: Optional[List[str]] = None,
    ) -> Document:
        """
        
        Perform document cleaning on a single document and return a single document. This method will deal with whitespaces, headers, footers
        and empty lines. Its exact functionality is defined by the parameters passed into PreProcessor.__init__().
        """
        if id_hash_keys is None:
            id_hash_keys = self.id_hash_keys

        if isinstance(document, dict):
            document = Document.from_dict(document, id_hash_keys=id_hash_keys)

        # Mainly needed for type checking
        if not isinstance(document, Document):
            raise HaystackError("Document must not be of type 'dict' but of type 'Document'.")
        text = document.content
        text = self.custom_preprocessor(text)
        if clean_header_footer:
            text = self._find_and_remove_header_footer(
                text, n_chars=300, n_first_pages_to_ignore=1, n_last_pages_to_ignore=1
            )

        if clean_whitespace:
            lines = text.splitlines()

            cleaned_lines = []
            for line in lines:
                line = line.strip()
                cleaned_lines.append(line)
            text = "\n".join(cleaned_lines)

        if clean_empty_lines:
            text = re.sub(r"\n\n+", "\n\n", text)

        for substring in remove_substrings:
            text = text.replace(substring, "")

        if text != document.content:
            document = deepcopy(document)
            document.content = text

        return document
    
    

In [151]:
from  functools import reduce
from operator import iconcat

In [152]:
all_cd_news_docs[0]

<Document: {'content': 'Les membres de la Commission tarifaire viennent de proposer des mesures nécessaires visant à corriger les erreurs matérielles pour faciliter la mise en œuvre de la taxe à valeur ajoutée sociale (TVA) et à l’extension du bénéfice du taux réduit à certaines positions tarifaires se rapportant aux produits à vocation sociale et de grande consommation ciblés notamment les produits laitiers pour nourrissons. C’est le résultat des travaux de 4 jours (du 03 au 06 mai dernier) des membres de la Commission tarifaire clôturés le vendredi 06 mai au Romeo Golf de la Gombe par Liévin Chiribagula, conseiller fiscal et représentant du ministre des Finances.', 'content_type': 'text', 'score': None, 'meta': {'posted_at': '2022-09-05 00:00:00'}, 'embedding': None, 'id': '5150ad41ae7d0d36c7fa0b291dbe704b'}>

With our document indexted int the elastic searh we can search , use the piaf dataset which have question with answers without paragraph and leverage them.

In [153]:
from collections import deque

In [154]:
import re
from gensim.utils import deaccent
from unicodedata import normalize as unicode_normalize

In [155]:
def replace_point(document):
    """replace the point with the wwt.www with space point before tokenizing the document .
    TOdos : this may have a a downside when the point is in the middle of a words
    Args:
        document (_type_): _description_
    """
    result = re.sub(r"(\S)\.(\S)", r"\1 . \2", document)
    return result

def replace_website_name(document):
    """sometimes the doucment has the name politico.cd or 7sur7.cd or actualite.cd, we would like to replace them by the 
    actual name of the website. before proper cleaning

    Args:
        document (_type_): _description_
    """
    # @TODO : not sure if this will work but , way better replace by the first line of match.
    
    result = re.sub(r"7SUR7.CD|politico.cd|actualite.cd|mediacongo.net", r"SITE_WEB", document, flags=re.IGNORECASE)
    return result

def remove_accents(document):
    input_without_accent = deaccent(document)
    return input_without_accent

def pre_clean_document(document):
    """pre clean the document by removing the accents and replacing the point with the wwt.www with space point before tokenizing the document .
    TOdos : this may have a a downside when the point is in the middle of a words
    and any other side of cleaning that we want to do .
    Args:
        document (_type_): _description_
    """
    result = remove_accents(document)
    result =  replace_website_name(result)
    result = replace_point(result)
    result = re.sub(r"This post has already been read \d+ times!", "", result) # remove unwanted text
    result = unicode_normalize("NFKD", result)
    return result

In [156]:
preprocessor = CustomPreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_respect_sentence_boundary=True,
    language="fr",
    custom_preprocessor=pre_clean_document,
)


cd_news_docs = preprocessor.process(all_cd_news_docs)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140235/140235 [07:51<00:00, 297.51docs/s]


In [157]:

print(f"\nn_docs_output: {len(cd_news_docs)}")


n_docs_output: 299449


In [158]:
write_documents_parallel(document_store, cd_news_docs)

I am inside the function


  0%|                                                                                                                                                                                                                                                                  | 0/30 [00:00<?, ?it/s]/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

#### Retrieving Documents

In [28]:
%%script false --no-raise-error
write_documents_parallel(document_store, scan_results)

#### PIAF Dataset

In [29]:
from haystack.nodes import BM25Retriever

In [30]:
bm25_retriever = BM25Retriever(document_store=document_store, all_terms_must_match=False)

In [31]:
import pandas as pd

In [33]:
from pathlib import Path
DATA_PATH = Path.cwd().parent.joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"

In [38]:
piaf_file

PosixPath('/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/corpus/raw/piaf/question-reponse.csv')

In [ ]:
/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/corpus/raw/piaf/question-reponse.csv

In [187]:
piaf_file_without_context = DATA_PATH.joinpath("corpus", "raw", "piaf", "question-reponse.csv")

In [39]:
assert piaf_file.exists(), "the piaf dataset does not exist"

piaf_question = data

In [40]:
piaf_df_without_context = pd.read_csv(piaf_file_without_context)

In [41]:
sample_question_response = piaf_df_without_context.sample(1)
question = deaccent(sample_question_response.question.values[0])
response = deaccent(sample_question_response.reponse.values[0])


In [42]:
question

"Quel est le principal sponsor de l'equipe cycliste Vacansoleil-DCM?"

In [273]:
def get_positive_context(retriever: BM25Retriever, search_query:str, answer:str, positive_documents: int = 100) -> List[Document]:    
        """given entitity retrieve the positive context
        we will first retrieve the top  100 documents , 
        - if the answer is in the top 40 document the input of the reader is the top 40 documents
        if the top 40 documents does not contain the answer we check whithin the top 41 to 100 document if the anwer is ther and we put it ther.
        other wise we discard the sentence

        Args:
            retriever (BM25Retriever): _description_
            n_ctxs (int, optional): _description_. Defaults to 15.
            entity (Entity, optional): _description_. Defaults to None.
        """
        list_pos_ctxs = []
        retrieved_docs = retriever.retrieve(query=search_query, top_k=positive_documents)
        for index, retrieve_doc in enumerate(retrieved_docs[0:40]):
            if deaccent(answer.lower()) in retrieve_doc.content.lower():
                list_pos_ctxs.append(
                    {"title": retrieve_doc.meta.get("title"), "content": retrieve_doc.content}
                )
        if len(list_pos_ctxs) == 0:
            for index, retrieve_doc in enumerate(retrieved_docs[40:100]):
                if deaccent(answer.lower()) in retrieve_doc.content.lower():
                    list_pos_ctxs.append(
                        {"title": retrieve_doc.meta.get("title"), "content": retrieve_doc.content}
                    )
        else:
            pass
        if len(list_pos_ctxs) == 0:
            return []
        return list_pos_ctxs

#### Use Piaf Dataset to query context

In [45]:
from datasets import load_dataset

In [46]:
piaf_dataset = load_dataset("piaf")

WARNING - datasets.load -  Using the latest cached version of the module from /Users/es.py/.cache/huggingface/modules/datasets_modules/datasets/piaf/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae (last modified on Thu Jul 14 08:39:15 2022) since it couldn't be found locally at piaf., or remotely on the Hugging Face Hub.
WARNING - datasets.builder -  Reusing dataset piaf (/Users/es.py/.cache/huggingface/datasets/piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae)


  0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
piaf_dataset = piaf_dataset["train"]

In [54]:
retrieved_docs

[{'title': 'Thurgood Marshall United State Courthouse',
  'content': "Le Thurgood Marshall United States Courthouse est un gratte-ciel de style neoclassique de 180 metres de hauteur construit a New York de 1933 a 1936 qui abrite un tribunal. L'immeuble a ete concu par l'architecte Cass Gilbert, le concepteur du Woolworth Building."},
 {'title': 'Architecture à New York',
  'content': "Le Woolworth Building, œuvre de l'architecte Cass Gilbert (1913), avec ses 60 etages, depassait alors la Metropolitan Life Tower. Les trois premiers niveaux sont pares d'un beau calcaire remplace aux niveaux suivants par de la terre cuite. La tendance neogothique a pousse l'architecte a ajouter des faux contreforts et des gargouilles. Compte tenu du gigantisme de l'edifice, les elements decoratifs ont ete surdimensionnes afin d'etre apercus depuis la rue. En 1924, Raymond Hood s'occupe de l'American Radiator Building de New York qu'il habille de couleurs et qu'il coiffe d'une decoration de terre cuite dor

In [183]:
%%script false --no-raise-error
for index in tqdm(piaf_df_without_context.index):
    question = piaf_df_without_context.loc[index].question
    response = piaf_df_without_context.loc[index].reponse
    question = deaccent(question)
    response = deaccent(response)
    retrieved_docs = get_positive_context(retriever=bm25_retriever, search_query=question, answer=response, positive_documents=100)
    piaf_df_without_context.loc[index, "positive_context"] = json.dumps(retrieved_docs)

By using our wiki corpus we are able to find some question with positive context, we will leverage them to build our qa system

In [188]:
piaf_with_context = piaf_df_without_context.loc[piaf_df_without_context.positive_context.apply(lambda x: len(json.loads(x))) > 0]

In [195]:
piaf_df_without_context = piaf_df_without_context.loc[piaf_df_without_context.positive_context.apply(lambda x: len(json.loads(x))) == 0]

In [196]:
piaf_df_without_context.head()

,question,reponse,positive_context
2,Comment fût payé le bâtiment commandé par Franck Woolworth?,en cash,[]
5,Quelle femme devint reine aux côtés de Philippe le Bel ?,Jeanne Ire de Navarre,[]
8,Quelle raison pousse Philippe Le Bel à organiser les premiers Etats généraux ?,pour lever de nouveaux impôts,[]
9,Quel souverain utilise les dévaluations monétaires pour s'enrichir ?,Philippe IV le Bel,[]
10,Quel souverain rattacha le compté de Toulouse au royaume ?,Philippe III le Hardi,[]


In [202]:
piaf_with_context = piaf_with_context.drop_duplicates(subset=["question", "reponse"])

with our dataset , we can find that {{piaf_with_context.shape[0]}} have positive context and that will be usefull to fine tune our model.

In [67]:
piaf_with_context = piaf_with_context.assign(length_positive_context = piaf_with_context["positive_context"].apply(lambda x: len(json.loads(x))))

In [68]:
piaf_with_context = piaf_with_context.loc[piaf_with_context.length_positive_context > 1].sort_values(by="length_positive_context", ascending=False)

In [390]:
piaf_with_context["contexts"] = piaf_with_context.positive_context.apply(lambda x: json.loads(x))

/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/1143678093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  piaf_with_context["contexts"] = piaf_with_context.positive_context.apply(lambda x: json.loads(x))


In [393]:
piaf_with_context.index.name = "id"

In [399]:
piaf_with_context = piaf_with_context.reset_index().loc[:, ["id", "question", "reponse", "contexts"]].rename({"reponse":"answer"}, axis="columns")

In [400]:
piaf_with_context.head()

,id,question,answer,contexts
0,0,Quel architecte fût à l'origine des plans du Woolworth building?,Cass Gilbert,"[{'title': 'Thurgood Marshall United State Courthouse', 'content': 'Le Thurg..."
1,1,Où se trouvait Franck Woolworth lors de l'inauguration de son immeuble New Y...,Washington,"[{'title': 'Rue Washington', 'content': ' 38-44 (et 29-31, rue de Berri) : i..."
2,3,En quelle année ouvrit le Woolworth Building ?,1913,"[{'title': 'Kay Johnson', 'content': 'Nee Katharine Johnson, elle etait la f..."
3,4,Qui commanda la construction du Woolworth Building ?,Frank Woolworth,"[{'title': 'Barbara Hutton', 'content': 'Edna Hutton se suicida alors que Ba..."
4,6,Quel créancier du roi fut supprimé en 1312 ?,l'ordre du Temple,"[{'title': 'Couvent de l'ordre du Christ', 'content': 'L'ordre du Temple a e..."


In [384]:
piaf_with_context_from_wikipedia = DATA_PATH.joinpath("corpus", "french-qa", "piaf-with-from-wikipedia-bm25")

In [385]:
piaf_with_context_from_wikipedia.mkdir(exist_ok=True)

In [402]:
for element in piaf_with_context.iterrows():
    key = element[0]
    with open(piaf_with_context_from_wikipedia.joinpath(f"{key}.json"), "w") as f:
        element[1].to_json(path_or_buf=f, orient="index", force_ascii=False, indent=4)

In [203]:
piaf_with_context.to_csv(DATA_PATH.joinpath("corpus", "raw", "piaf", "piaf_with_context.csv"))

In [ ]:
DATA_PATH.joinpath("corpus", "french-qa",

In [204]:
piaf_with_context.shape

(5565, 3)

within our dataset , 5550 row have more than one context.

At this point , we have a dataset with context and we have contributed to something

### Adding Additional Context to PIAF

With our dataset with context, let us pull more question with context for the original piaf dataset with context

In [72]:
piaf_dataset = load_dataset("piaf")

WARNING - datasets.load -  Using the latest cached version of the module from /Users/es.py/.cache/huggingface/modules/datasets_modules/datasets/piaf/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae (last modified on Thu Jul 14 08:39:15 2022) since it couldn't be found locally at piaf., or remotely on the Hugging Face Hub.
WARNING - datasets.builder -  Reusing dataset piaf (/Users/es.py/.cache/huggingface/datasets/piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae)


  0%|          | 0/1 [00:00<?, ?it/s]

In [75]:
piaf_df = piaf_dataset.to_pandas()

In [76]:
piaf_df.head()

,id,title,context,question,answers
0,p140295443291664,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Combien de personnes travaillent au ministère des sports,"{'text': ['100 000'], 'answer_start': [472]}"
1,p140295443291520,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Combien d'employeurs,"{'text': ['20 000'], 'answer_start': [597]}"
2,p140295443291376,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel part du budget des ménages,"{'text': ['50'], 'answer_start': [46]}"
3,p140295443291088,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel montant en 2003,"{'text': ['14,2 milliards'], 'answer_start': [68]}"
4,p140295443290872,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel montant en 2019,"{'text': ['12 milliards'], 'answer_start': [102]}"


What are we trying to achive with this ?


We have question on the piaf dataset with context, for those question we will query elastic search to find out  the additional context. That context will will be consider as additional context to train our model with.


The algo : 

- for each question , query the elastic search and keep the top 10 documents context
- we will loop and send queries in a batch of 10 questions to elastic search
- retrieved the context and and save anything on the disk.

In the future to improve the quality of our finding we can consider only documents with named entities in the answers. We can check the paragraph , run the NER model on it and then check if the answer is in the named entities.

In [77]:
sample_questions = np.vectorize(deaccent)(piaf_df.question.loc[1:5])
retrieved_docs = bm25_retriever.retrieve_batch(queries=sample_questions.tolist(), top_k=30)

/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [205]:
async def query_batch(retriever, queries, top_k=10):
    return retriever.retrieve_batch(queries=queries, top_k=top_k)

In [206]:
def decent_vectorize(queries):
    return np.vectorize(deaccent)(queries)

In [207]:
question_chunk = [decent_vectorize(piaf_df.question.loc[1:5]), decent_vectorize(piaf_df.question.loc[5:10])]

In [208]:
question_chunk

[array(["Combien d'employeurs", 'Quel part du budget des menages',
        'Quel montant en 2003', 'Quel montant en 2019',
        'En quelle annee Jakob Bohme tombe-t-il malade ?'], dtype='<U47'),
 array(['En quelle annee Jakob Bohme tombe-t-il malade ?',
        'Qui est mort en juillet ?',
        'Quel est le metier de Nicolas Thomas ?',
        'Que doit subir Jakob Bohme avant d’obtenir les derniers sacrements ?',
        'Pourquoi Bohme enfle-t-il ?',
        'Quel pays est surnomme la "perle de l\'Afrique" ?'], dtype='<U68')]

In [209]:
async def main():
    return await tqdm_asyncio.gather(*[query_batch(bm25_retriever, queries) for queries in question_chunk])

In [210]:
def write_to_json(data, path):
    with open(path, "w") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [211]:
question_chunk[0][-1]

'En quelle annee Jakob Bohme tombe-t-il malade ?'

In [212]:
piaf_dataset[0]

{'id': 'p140295443291664',
 'title': 'Sport',
 'context': "Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 milliards d'euros en 2003 et 12 milliards d'euros en 2019), contre 7,9 milliards d'euros pour les collectivités locales, 3,2 pour l'État, et 2,2 pour les entreprises. Parmi les dépenses sportives des ménages en 2003, 3,7 milliards sont consacrés aux vêtements de sport et chaussures, 2 aux biens durables, 2,7 aux autres biens et 5,8 aux services. Le Ministère de la Jeunesse et des Sports estime à 100 000 (58 % d'hommes pour 42 % de femmes) le nombre de salariés travaillant pour le secteur sportif en France pour quelque 20 000 employeurs.",
 'question': 'Combien de personnes travaillent au ministère des sports',
 'answers': {'text': ['100 000'], 'answer_start': [472]}}

In [213]:
import json

In [234]:
def save_to_json(instances, path):
    "saves an iterator of multiple json files to files in the path directory"
    for instance in instances:
        with open(path.joinpath(f"{instance['id']}.json"), "w") as f:
            json.dump(instance, f, indent=4, ensure_ascii=False)

In [434]:
async def process_queries_chunk(retriever:BM25Retriever, queries_chunk: pd.DataFrame, validate_answers, path):
    """tak a query chunk process it , query the elastic, instance and write the document to the file.

    Args:
        queries_chunk (pd.DataFrame): _description_
    """
    questions = decent_vectorize(queries_chunk.question)
    titles = queries_chunk.title
    answers = [answer.get("text")[0] for answer in queries_chunk.answers]
    contexts = queries_chunk.context
    ids = queries_chunk.id
    retrieved_docs = retriever.retrieve_batch(queries=questions, top_k=10)
    instances = process_answers(ids=ids, 
                    questions=questions,
                    titles=titles,
                    answers=answers,
                    contexts=contexts,
                    retrieved_docs=retrieved_docs,
                    validate_answers=validate_answers)
    save_to_json(instances, path)
    

In [440]:
def process_doc(retrieved_docs, answer, main_doc, validate_answers):
    """
    convert a list of retrieved document to list of array of list and content
    """
    doc_list = []
    for doc in retrieved_docs:
        content = deaccent(doc.content)
        if content != deaccent(main_doc):
            if validate_answers:
                if answer in content:
                    doc_ = {
                        "title": doc.meta.get("title"),
                        "content": content,
                        }
                    doc_list.append(doc_)
                else:
                    pass
            else:
                doc_ = {
                        "title": doc.meta.get("title"),
                        "content": content,
                        }
                doc_list.append(doc_)
        else:
            pass
    return doc_list

In [421]:
piaf_with_multi_context_supporting = DATA_PATH.joinpath("corpus", "french-qa", "piaf-with-multi-context-with-or-without-answer")
piaf_with_multi_context_supporting.mkdir(exist_ok=True)

In [431]:
def process_answers(ids, questions, titles, answers, contexts, retrieved_docs, validate_answers):
    """process answers and write them to the file.

    Args:
        questions (_type_): _description_
        titles (_type_): _description_
        answers (_type_): _description_
        contexts (_type_): _description_
        retrieved_docs (_type_): _description_
    """
    for id_, question, title, answer, context, retrieved_doc in zip(ids, questions, titles, answers, contexts, retrieved_docs):
        contexts = [{"title": title, "content": context}]
        additional_context = process_doc(retrieved_doc, answer, context, validate_answers)
        contexts.extend(additional_context)
        instance_json = {
            "question": question,
            "answer": answer,
            "contexts": contexts,
            "id": id_,
        }
        yield instance_json

    

In [438]:
async def main():
    await tqdm_asyncio.gather(*[process_queries_chunk(bm25_retriever, queries, False, piaf_with_multi_context_supporting) for _, queries in piaf_df.groupby(np.arange(len(piaf_df))//5)])

In [441]:

await main()




  0%|                                                                          | 0/767 [00:00<?, ?it/s]/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)



100%|████████████████████████████████████████████████████████████████| 767/767 [02:11<00:00,  5.85it/s]


###  Adding Additional Context to Fquad

In [347]:
frquad_path_train = DATA_PATH.joinpath("corpus", "french-qa", "fquad", "train.json")
frquad_path_valid = DATA_PATH.joinpath("corpus", "french-qa", "fquad", "valid.json")

In [348]:
assert frquad_path_train.exists()
assert frquad_path_valid.exists()

In [349]:
with open(frquad_path_train, "r") as f:
    frquad_train = json.load(f)

In [350]:
frquad_train = frquad_train.get("data")

In [351]:
with open(frquad_path_valid, "r") as f:
    frquad_valid = json.load(f)

In [352]:
frquad_valid = frquad_valid.get("data")

In [353]:
fquad_path = DATA_PATH.joinpath("corpus", "french-qa", "fquad-with-multi-context")
fquad_path_output_train = fquad_path.joinpath("train")
fquad_path_output_valid = fquad_path.joinpath("valid")

In [427]:
fquad_path_with_supporting = DATA_PATH.joinpath("corpus", "french-qa", "fquad-with-multi-context-with-without-answers")
fquad_path_output_supporting_train = fquad_path_with_supporting.joinpath("train")
fquad_path_output_supporting_valid = fquad_path_with_supporting.joinpath("valid")

In [426]:
fquad_path_with_supporting.mkdir(exist_ok=True)

In [354]:
for doc in frquad_train[0:2]:
    print(doc.get("title"))

(1)-cérès
american-idiot


In [446]:
async def process_fquad_paragraph(retriever, paragraph, title, validate_answers,path ):
    """
    this will loop over the  a paragraph in frquad dataset.
    since each paragraph have a list of question ,
     it will return query the elasticsearch for each batch of question 
    and return the additional related_context

    Args:
        paragraph (_type_): _description_
    """
    context  = paragraph.get("context")
    question_answers = paragraph.get("qas")
    ids = [qa.get("id") for qa in question_answers]
    questions = [qa.get("question") for qa in question_answers]
    answers = [qa.get("answers")[0].get("text") for qa in question_answers]
    retrieved_docs = retriever.retrieve_batch(queries=questions, top_k=10)
    instances = process_answers(ids=ids, 
                                questions=questions,
                                titles=repeat(title, len(questions)),
                                answers=answers,
                                contexts=repeat(context, len(questions)),
                                retrieved_docs=retrieved_docs,
                                validate_answers=validate_answers)
    save_to_json(instances, path)

In [447]:
assert fquad_path.exists()

AssertionError: 

In [448]:
def check_answer_in_retrieved_docs(answer, retrieved_docs):
    for doc in retrieved_docs:
        if answer in doc.content:
            print(doc)
            return True
    return False

In [453]:
async def process_fquad(fquad, path, retriever):
    all_article_processor = list()
    for document in tqdm_asyncio(fquad):
        title = document.get("title")
        paragraphs = document.get("paragraphs")
        doc_coroutine = tqdm_asyncio.gather(*[process_fquad_paragraph(retriever=bm25_retriever, paragraph=paragraph, title=title, validate_answers=False, path=path) for paragraph in paragraphs])
        all_article_processor.append(doc_coroutine)
    return await tqdm_asyncio.gather(*all_article_processor)

In [454]:
await process_fquad(frquad_train, fquad_path_output_supporting_train, bm25_retriever)




  0%|                                                                          | 0/117 [00:00<?, ?it/s]


100%|███████████████████████████████████████████████████████████████| 117/117 [00:00<00:00, 767.01it/s]



  0%|                                                                          | 0/117 [00:00<?, ?it/s]



  0%|                                                                           | 0/45 [00:00<?, ?it/s]




  0%|                                                                           | 0/20 [00:00<?, ?it/s]





  0%|                                                                           | 0/24 [00:00<?, ?it/s]






  0%|                                                                           | 0/66 [00:00<?, ?it/s]







  0%|                                                                           | 0/21 [00:00<?, ?it/s]








  0%|                                                                           | 0/64 [00:00<?, ?it/s]









  0%|         

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-43070' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=TypeError('Object of type Document is not JSON serializable')>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 5, in save_to_json
    json.dump(instance, f, indent=4, ensure_ascii=False)
  File "/usr/local/Cellar/python@3.9/3.9.7/Frameworks/Python.framework/Ver

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-43075' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=TypeError('Object of type Document is not JSON serializable')>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 5, in save_to_json
    json.dump(instance, f, indent=4, ensure_ascii=False)
  File "/usr/local/Cellar/python@3.9/3.9.7/Frameworks/Python.framework/Ver

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-43080' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=TypeError('Object of type Document is not JSON serializable')>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 5, in save_to_json
    json.dump(instance, f, indent=4, ensure_ascii=False)
  File "/usr/local/Cellar/python@3.9/3.9.7/Frameworks/Python.framework/Ver

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-43085' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=TypeError('Object of type Document is not JSON serializable')>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 5, in save_to_json
    json.dump(instance, f, indent=4, ensure_ascii=False)
  File "/usr/local/Cellar/python@3.9/3.9.7/Frameworks/Python.framework/Ver

...

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-42780' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=UnboundLocalError("local variable 'doc_' referenced before assignment")>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 3, in save_to_json
    for instance in instances:
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/1928996066.py", l

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-42787' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=UnboundLocalError("local variable 'doc_' referenced before assignment")>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 3, in save_to_json
    for instance in instances:
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/1928996066.py", l

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-42794' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=UnboundLocalError("local variable 'doc_' referenced before assignment")>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/3039580125.py", line 20, in process_queries_chunk
    save_to_json(instances, path)
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/2234439518.py", line 3, in save_to_json
    for instance in instances:
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/1928996066.py", l

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

ERROR - asyncio -  Task exception was never retrieved
future: <Task finished name='Task-44576' coro=<tqdm_asyncio.gather.<locals>.wrap_awaitable() done, defined at /Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py:75> exception=TypeError('Object of type Document is not JSON serializable')>
Traceback (most recent call last):
  File "/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/tqdm/asyncio.py", line 76, in wrap_awaitable
    return i, await f
  File "/var/folders/5_/n81dq93n79l30d_c34cfqxpr0000gn/T/ipykernel_23412/

FileNotFoundError: [Errno 2] No such file or directory: '/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/data/corpus/french-qa/fquad-with-multi-context-with-without-answers/train/b784f166-61d9-4251-a532-e484f7f12701.json'

In [ ]:
await process_fquad(frquad_valid, fquad_path_output_supporting_valid, bm25_retriever)

In [362]:
def save_doc_to_json(doc, file):
    "saves an iterator of multiple json files to files in the path directory"
    with open(file, "w") as f:
        json.dump(doc, f, indent=4, ensure_ascii=False)

In [363]:
def count_positive_context(path):
    json_files = path.glob('**/*.json')
    document_with_more = 0
    for file in json_files:
        with open(file, "r") as buffer:
            json_doc = json.load(buffer)
            contexts = json_doc["contexts"]
            if len(contexts) >1 :
                document_with_more +=1
    return document_with_more

In [364]:
count_piaf = count_positive_context(piaf_with_multi_context_path)

In [365]:
count_train_fquad = count_positive_context(fquad_path_output_train)

In [366]:
count_valid_fquad = count_positive_context(fquad_path_output_valid)

In [367]:
count_train_fquad

7418

In [368]:
count_valid_fquad

1130

In [369]:
count_piaf

1134

### Exetat_questions

At this point we have the piaf dataset with context, we have the frquad with context, let us look now the exetat questions with context.

In [264]:
exetat_questions_path = DATA_PATH.joinpath("corpus", "french-qa", "exetat-questions", "questions.json")

In [265]:
assert exetat_questions_path.exists()

In [266]:
with open(exetat_questions_path, "r") as f:
    exetat_questions = json.load(f)

In [267]:
len(exetat_questions)

173

for now we can say that we have 173 exams questions, we need more and get more context to train.frquad_train
But for now we are going to train our model on the conbinaision of piaf and frenchquad dataset.

In [274]:
def process_exetat_question(retriever, questions):
    """
    retrieve paragraph containing the answers from the extetat question

    Args:
        paragraph (_type_): _description_
    """
    processed_questions = []
    for data in tqdm(questions):
        question = data.get("question")
        correct_index = int(data.get("correct_index"))
        options = data.get("options")
        answer = options[correct_index]
        contexts = get_positive_context(retriever, question, answer, positive_documents=300)
        processed_object = {"question": question, "answer": answer, "contexts": contexts}
        processed_questions.append(processed_object)
    return processed_questions
        
        

In [275]:
exetat_questions_processed = process_exetat_question(bm25_retriever, exetat_questions)


  0%|                                                   | 0/173 [00:00<?, ?it/s]

Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Indiquez l'attribution qui est exclusivement réservée au parlement", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}



  1%|▍                                          | 2/173 [00:00<00:31,  5.46it/s]

Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': 'Indiquez les éléments constitutifs du nationalisme', 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}
Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Le Facilitateur Eden Kodjo désigné par l'Union Africaine (U.A.) en vue d'amorcer le dialogue national inclusif en R.D.C. est de nationalité", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}



  2%|▋                                          | 3/173 [00:00<00:38,  4.40it/s]

...

Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Indiquez la province de la République Démocratique du Congo qui contient à la fois les gisements diamantifères et l'or.", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}
Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Indiquez l'alliage de l'étain utilisé dans les conserveries.", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}



 99%|████████████████████████████████████████▌| 171/173 [00:43<00:00,  5.49it/s]

Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Le rendement en métal pur d'uranium naturel est de:", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}
Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Parmi les pays de culture vivrière et industrielle, indiquez celui qui est originaire de l'Afrique.", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}



100%|█████████████████████████████████████████| 173/173 [00:43<00:00,  3.94it/s]

Retriever query: {'size': '300', 'query': {'bool': {'must': [{'multi_match': {'query': "Indiquez le premier pays producteur mondial d'uranium.", 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}


In [276]:
exetat_with_answers = list(filter(lambda x: len(x["contexts"]) >0, exetat_questions_processed))
exetat_without_anwers = list(filter(lambda x: len(x["contexts"]) == 0, exetat_questions_processed))

In [277]:
for element in piaf_with_context.iterrows():
    key = element[0]
    with open(piaf_with_context_from_wikipedia.joinpath(f"{key}.json"), "w") as f:
        element[1].to_json(path_or_buf=f, orient="index", force_ascii=False, indent=4)

35

Only 27 question from our extetat dataset have an answer after querying using bm25

In [404]:
exetat_with_context_bm25 = exetat_questions_path.parent.parent.joinpath("exetat-with-context-from-wikipedia-bm25")

In [408]:
exetat_with_context_bm25.mkdir(exist_ok=True)

In [413]:
for key, element in enumerate(exetat_with_answers):
    with open(exetat_with_context_bm25.joinpath(f"{key}.json"), "w") as f:
        json.dump(element, f, ensure_ascii=False, indent=4)

In [286]:
with open(exetat_questions_path.parent.joinpath("exetat-with-answers.json"), "w+") as buffer:
    json.dump(exetat_with_answers, buffer)

In [287]:
with open(exetat_questions_path.parent.joinpath("exetat-without-answers.json"), "w+") as buffer:
    json.dump(exetat_without_anwers, buffer)